# Setup

In [ ]:
# Parameters

## Years
start_year = 2015
end_year = 2016
base_year = start_year
years = range(start_year, end_year + 1)

## Grouping
young_age_cutoff=25
old_age_threshold=65

## Indexing
price_variable = 'mehir' # 'mehir' or 'omdan'

## Output
top_n = 10
comparison_year = end_year
comparison_level = 'primary'

## Folder Names
cex_data_folder="/Users/roykisluk/Downloads/Consumer_Expenditure_Survey/"
folder_names_pathname='Data_clean/CEX_folder_names.csv'
age_groups_pathname='Data_clean/age_groups.csv'
prodcode_dict_pathname = 'Data_clean/prodcode_dictionary_c3-c399.csv'
    
## Libraries
import pandas as pd
import pyreadstat  
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Data

In [ ]:
# Load folder names
folder_names_df = pd.read_csv(folder_names_pathname)

# Load age groups
age_groups_df = pd.read_csv(age_groups_pathname)
young_age_group_id = age_groups_df[(age_groups_df['min_age'] <= young_age_cutoff) & (age_groups_df['max_age'] >= young_age_cutoff)].index[0] + 1
old_age_group_id = age_groups_df[(age_groups_df['min_age'] <= old_age_threshold) & (age_groups_df['max_age'] >= old_age_threshold)].index[0] + 1

# Load household data for each year
dfs_mb = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_HH_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datamb.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_HH_pathname)
    df.columns = df.columns.str.lower()
    if 'gil' in df.columns:
        df.rename(columns={'gil': 'age_group'}, inplace=True)
    df['misparmb'] = df['misparmb'].astype(int)
    dfs_mb[year] = df

# Load individual data for each year
dfs_prat = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_IND_pathname = f"{cex_data_folder}{subfolder}/{subfolder}dataprat.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_IND_pathname)
    df.columns = df.columns.str.lower()
    if 'gil' in df.columns:
        df.rename(columns={'gil': 'age_group'}, inplace=True)
    df['misparmb'] = df['misparmb'].astype(int)
    dfs_prat[year] = df

# Load expenses data for each year
dfs_prod = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_prices_pathname = f"{cex_data_folder}{subfolder}/{subfolder}dataprod.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_prices_pathname)
    df.columns = df.columns.str.lower()
    df['misparmb'] = df['misparmb'].astype(int)
    df['prodcode'] = df['prodcode'].astype(int).astype(str)
    dfs_prod[year] = df

# Load survey data for each year
dfs_survey = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_prices_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datayoman.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_prices_pathname)
    df.columns = df.columns.str.lower()
    df['misparmb'] = df['misparmb'].astype(int)
    df['prodcode'] = df['prodcode'].astype(int).astype(str)
    dfs_survey[year] = df

# Grouping

## Data

In [ ]:
Groups = {}
for year in years:
    Groups[year] = pd.DataFrame(dfs_mb[year]['misparmb'].unique(), columns=['misparmb'])

In [ ]:
for year in years:
    dfs_mb_year = dfs_mb[year]
    dfs_prat_year = dfs_prat[year]

    nationality_map = {1: 'Jewish', 2: 'Arab'}
    observance_map = {1: 'Secular', 2: 'Conservative', 3: 'Religious', 4: 'Ultra-Orthodox', 5: 'Mixed'}


    Groups[year]['Nationality'] = dfs_mb_year['nationality'].map(nationality_map).fillna('Other')
    Groups[year]['Observance'] = dfs_mb_year['ramatdatiyut'].map(observance_map).fillna('Other')
    Groups[year].loc[Groups[year]['Nationality'] == 'Arab', 'Observance'] = 'Other'

    age_group_map = {age_group_id: '(1) Young' if age_group_id <= young_age_group_id else '(3) Old' if age_group_id >= old_age_group_id else '(2) Middle' for age_group_id in dfs_prat_year['age_group'].unique()}
    Groups[year]['Age_Group'] = dfs_prat_year.loc[dfs_prat_year['y_kalkali'] == 1, 'age_group'].map(age_group_map).values

    Groups[year]['Income_Decile'] = dfs_mb_year['decile'].fillna(0).astype(int)

    Groups[year]['Income_Quintile'] = pd.cut(dfs_mb_year['decile'], bins=[0, 2, 4, 6, 8, 10], labels=[1, 2, 3, 4, 5])

    Groups[year]['SES_Quintile'] = dfs_mb_year['cluster'].apply(lambda x: x if x in range(1, 6) else np.nan).fillna(0).astype(int)
    Groups[year]['SES_Groups'] = Groups[year]['SES_Quintile'].apply(lambda x: '(1) Low' if x in [1, 2] else '(2) Medium' if x == 3 else '(3) High' if x in [4, 5] else np.nan)

    Groups[year]['Children'] = dfs_mb_year['nefashotad18'].fillna(0).astype(int)
    Groups[year]['Family_Size'] = Groups[year]['Children'].apply(lambda x: '0 children' if x == 0 else '1-3 children' if x in [1, 2, 3] else '4+ children')

## Groups Dataframes Headers

In [ ]:
display(HTML(f"<h2>Groups for Year {end_year}</h2>"))
display(HTML(Groups[year].head().to_html(index=False)))
print(f"Number of observations: {len(dfs_mb[end_year])}")

## Plot Groups Distribution

In [ ]:
import seaborn as sns

# Get the columns to plot
columns_to_plot = [col for col in Groups[end_year].columns if col != 'misparmb']

# Calculate the number of rows needed
ncols = 3
nrows = (len(columns_to_plot) + ncols - 1) // ncols

# Create subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Define a color palette
palette = sns.color_palette("husl", len(columns_to_plot))

# Plot each column
for ax, column, color in zip(axes, columns_to_plot, palette):
    Groups[end_year][column].value_counts().sort_index().plot(kind='bar', ax=ax, color=color)
    ax.set_title(f'Distribution of {column} in {end_year}')
    ax.set_xlabel(column)
    ax.set_ylabel('Count')

# Remove any unused subplots
for ax in axes[len(columns_to_plot):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.show()

# Indexing

## Laspeyres Index

For good $j$, at time $t$:

$$
I_{t}=\frac{\sum_{j\in L}{\frac{P_{tj}}{P_{oj}}(P_{oj}Q_{oj})}}{\sum_{j\in L}P_{oj}Q_{oj}}\times 100
$$

$$\text{For our purposes:}$$

$$
I_{tj}=\frac{P_{tj}}{P_{oj}}
$$
$$
W_{oj}=\frac{P_{oj}Q_{oj}}{\sum_{j\in L}P_{oj}Q_{oj}}
$$
$$
I_{t}=\sum_{j\in L}W_{oj}I_{tj}\times 100
$$



$$
\text{Where:}\\
I_{t}\text{  - Index for period t}\\
Q_{oj}\text{  - Quantity of the good or service in the base period}\\
P_{oj}\text{  - Price of the good or service in the base period}\\
P_{tj}\text{  - Price of the good or service in period t}\\
L\text{  - The set of all goods and services in the index basket}\\
$$

## Calculate Weights

In [ ]:
# Drop unnecessary groups
for year in years:
    Groups[year] = Groups[year].drop(columns=['Income_Decile', 'SES_Quintile', 'Children'])

# Drop unnecessary subgroups
for year in years:
    Groups[year] = Groups[year][~Groups[year]['Observance'].isin(['Mixed'])]
    Groups[year] = Groups[year][Groups[year]['Nationality'] != 'Other']

In [ ]:
def calculate_weights(product_level, year, group_mmb = None):
    # Expenses dataframe for consumption expenses only
    expenses_df = dfs_prod[year][dfs_prod[year]['prodcode'].astype(str).str.startswith('3')].copy()

    if group_mmb is not None:
        # Filter only IDs that match the group
        expenses_df = expenses_df[expenses_df['misparmb'].isin(group_mmb)].reset_index(drop=True)

    # Keep only the product codes at the correct product level
    expenses_df = expenses_df[expenses_df['prodcode'].str.len() == product_level]

    # Sum the expense for each prodcode
    expenses_df = expenses_df.groupby('prodcode')['schum'].sum().reset_index()

    # Calculate weights
    expenses_df['weight'] = expenses_df['schum'] / expenses_df['schum'].sum()

    return expenses_df

In [ ]:
product_level = 6
year = base_year

weights = {}
for group in Groups[year].columns[1:]:
    weights[group] = {}  
    for subgroup in Groups[year][group].unique():
        mmb = Groups[year][Groups[year][group] == subgroup]['misparmb']
        weights[group][subgroup] = calculate_weights(product_level, year, mmb)
weights['All'] = {'All': calculate_weights(product_level, year, None)}

In [ ]:
# Check if the weights sum to 1.0
for group in weights:
    for subgroup in weights[group]:
        if weights[group][subgroup]['weight'].sum()!=1.0:
            print(f"Warning: Weights for {group} {subgroup} do not sum to 1.0. They sum to {weights[group][subgroup]['weight'].sum()}")

In [ ]:
# Define the groups to plot
groups_to_plot = list(weights.keys())

# Calculate the number of rows needed
ncols = 3
nrows = (len(groups_to_plot) + ncols - 1) // ncols

# Create subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))
axes = axes.flatten()

# Plot each group
for ax, group in zip(axes, groups_to_plot):
    subgroups = weights[group].keys()
    for subgroup in subgroups:
        weight_values = weights[group][subgroup]['weight']
        ax.hist(weight_values, bins=30, alpha=0.5, label=subgroup)
    ax.set_title(f'Weight Distribution for {group}')
    ax.set_xlabel('Weight')
    ax.set_ylabel('Frequency')
    ax.legend()

# Remove any unused subplots
for ax in axes[len(groups_to_plot):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.show()

## Prices

In [ ]:
def calculate_prices(year, group_mmb = None):
    # Prices dataframe for consumption expenses only
    prices_df = dfs_survey[year][dfs_survey[year]['prodcode'].astype(str).str.startswith('3')].copy()

    if group_mmb is not None:
        # Filter only IDs that match the group
        prices_df = prices_df[prices_df['misparmb'].isin(group_mmb)].reset_index(drop=True)

    # Calculate prices
    prices_df['price'] = prices_df['mehir'] / prices_df['kamut']
    prices_df['price'] = prices_df['price'].replace([np.inf, 0], np.nan)

    # Group by product code and calculate the mean price, standard deviation, min and max
    prices_df = prices_df.groupby('prodcode', as_index=False).agg({'price': ['mean', 'std', 'min', 'max']})
    prices_df.columns = ['prodcode', 'price', 'price_std', 'price_min', 'price_max']    

    return prices_df

In [ ]:
prices = {}
for year in years:
    prices[year] = {}
    for group in Groups[year].columns[1:]:
        prices[year][group] = {}
        for subgroup in Groups[year][group].unique():
            mmb = Groups[year][Groups[year][group] == subgroup]['misparmb']
            prices[year][group][subgroup] = calculate_prices(year, mmb)
    prices[year]['All'] = {'All': calculate_prices(year, None)}

In [ ]:
def calculate_indexes(weights, prices, prices_base):
    
    # Organize dataframes
    prices_base = prices_base.drop(columns=['price_std', 'price_min', 'price_max'])
    prices = prices.drop(columns=['price_std', 'price_min', 'price_max'])
    prices_base = prices_base.rename(columns={'price': 'price_base'})
    weights = weights.drop(columns=['schum'])

    # Merge weights and prices dataframes
    merged_df = weights.merge(prices, on='prodcode', how='left')
    merged_df = merged_df.merge(prices_base[['prodcode', 'price_base']], on='prodcode', how='left')

    # Calculate price divided by the base year price
    merged_df['price_ratio'] = merged_df['price'] / merged_df['price_base']
    merged_df['price_ratio'] = merged_df['price_ratio'].fillna(1)

    return merged_df

In [ ]:
price_indexes = {}
for year in years:
    price_indexes[year] = {}
    for group in Groups[year].columns[1:]:
        price_indexes[year][group] = {}
        for subgroup in Groups[year][group].unique():
            if subgroup in weights[group] and subgroup in prices[year][group] and subgroup in prices[base_year][group]:
                price_indexes[year][group][subgroup] = calculate_indexes(weights[group][subgroup], prices[year][group][subgroup], prices[base_year][group][subgroup])
    if 'All' in weights and 'All' in prices[year] and 'All' in prices[base_year]:
        price_indexes[year]['All'] = {'All': calculate_indexes(weights['All']['All'], prices[year]['All']['All'], prices[base_year]['All']['All'])}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the groups to plot
groups_to_plot = list(price_indexes[comparison_year].keys())

# Calculate the number of rows needed
ncols = 3
nrows = (len(groups_to_plot) + ncols - 1) // ncols

# Create subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))
axes = axes.flatten()

# Plot each group
for ax, group in zip(axes, groups_to_plot):
    subgroups = price_indexes[comparison_year][group].keys()
    for subgroup in subgroups:
        price_ratios = price_indexes[comparison_year][group][subgroup]['price_ratio']
        ax.hist(price_ratios, bins=np.arange(0, 5.1, 0.1), alpha=0.5, label=subgroup)
    ax.set_title(f'Price Ratio Distribution for {group}')
    ax.set_xlabel('Price Ratio')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0, 3)
    ax.legend()

# Remove any unused subplots
for ax in axes[len(groups_to_plot):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.show()

## Laspeyres Group Indexes

In [ ]:
yearly_indexes = {}
for group in Groups[base_year].columns[1:]:
    # Initialize dataframe
    df = pd.DataFrame({'year': years})
    for subgroup in Groups[base_year][group].unique():
        df[subgroup] = None 
    for year in years:
        for subgroup in Groups[year][group].unique():
            if subgroup in price_indexes[year][group]:
                df.loc[df['year'] == year, subgroup] = (price_indexes[year][group][subgroup]['price_ratio'] * price_indexes[year][group][subgroup]['weight']).sum() * 100
    yearly_indexes[group] = df

# Add 'All' group
df_all = pd.DataFrame({'year': years})
df_all['all'] = None
for year in years:
    if 'All' in price_indexes[year] and 'All' in price_indexes[year]['All']:
        df_all.loc[df_all['year'] == year, 'all'] = (price_indexes[year]['All']['All']['price_ratio'] * price_indexes[year]['All']['All']['weight']).sum() * 100
yearly_indexes['All'] = df_all

## Merging to Secondary Categories

In [ ]:
def merge_to_secondary(df):
    grouped = df.copy()
    grouped['prodcode_secondary'] = grouped['prodcode'].astype(str).str[:3]
    grouped = grouped.groupby('prodcode_secondary', group_keys=False).apply(
        lambda x: pd.Series({
            'price_ratio': np.average(x['price_ratio'], weights=x['weight']) if x['weight'].sum() > 0 else np.nan,
            'total_weight': x['weight'].sum()
        }),
        include_groups=False 
    ).reset_index()
    grouped.rename(columns={'prodcode_secondary': 'prodcode'}, inplace=True)
    grouped.rename(columns={'total_weight': 'weight'}, inplace=True)
    return grouped

In [ ]:
# Merge to secondary categories
price_indexes_secondary = {}
for year in years:
    price_indexes_secondary[year] = {}
    for group in price_indexes[year].keys():
        price_indexes_secondary[year][group] = {}
        for subgroup in price_indexes[year][group].keys():
            price_indexes_secondary[year][group][subgroup] = merge_to_secondary(price_indexes[year][group][subgroup])

In [ ]:
# Load prodcode dictionary
prodcode_dict_df = pd.read_csv(prodcode_dict_pathname)
prodcode_dict_df['prodcode']=prodcode_dict_df['prodcode'].astype(str)

In [ ]:
# Add description to secondary categories
for year in price_indexes_secondary:
    for group in price_indexes_secondary[year]:
        for subgroup in price_indexes_secondary[year][group]:
            price_indexes_secondary[year][group][subgroup] = price_indexes_secondary[year][group][subgroup].merge(prodcode_dict_df, on='prodcode', how='left')

# Plots

## Price Index Over Time

In [ ]:
def price_index_over_time(yearly_indexes):
    import matplotlib.pyplot as plt

    # Calculate the number of groups
    n_groups = len(yearly_indexes) - 1
    ncols = 3
    nrows = (n_groups + ncols - 1) // ncols

    # Create subplots
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 5))
    axes = axes.flatten()

    # Plot each group
    for ax, (group, yearly_indexes_group) in zip(axes, yearly_indexes.items()):
        years = yearly_indexes_group['year']
        indexes = yearly_indexes_group.drop(columns=['year'])
        for subgroup in indexes.columns:
            if group == 'All' and subgroup == 'all':
                continue
            ax.plot(years, indexes[subgroup], label=subgroup)
            for i, year in enumerate(years):
                ax.text(year, indexes[subgroup].iloc[i], subgroup, fontsize=8, ha='right')
        
        # Add dotted line for 'All' group
        if 'all' in yearly_indexes['All'].columns:
            ax.plot(yearly_indexes['All']['year'], yearly_indexes['All']['all'], label='All', linestyle='dotted', color='black')
        
        ax.set_title(f"{group}: Price Index Over Time")
        ax.set_xlabel('Year')
        ax.set_ylabel('Price Index')
        ax.legend()
        ax.grid(True)

    # Remove any unused subplots
    for ax in axes[len(yearly_indexes):]:
        fig.delaxes(ax)

    plt.tight_layout()
    plt.show()

price_index_over_time(yearly_indexes)


## Top Weights

In [ ]:
from IPython.display import display, HTML

def display_top_weights(weights, top_n=10):
    for group in weights.keys():
        if group == 'All':
            continue

        subgroups = weights[group].keys()
        try:
            subgroups = sorted(subgroups, key=int)
        except ValueError:
            subgroups = weights[group].keys()

        for subgroup in subgroups:
            # Sort by the weight in descending order
            sorted_weights_df = weights[group][subgroup].sort_values(by='weight', ascending=False)

            # Select the top n weights
            top_n_weights_df = sorted_weights_df.head(top_n)

            # Display the HTML table
            display(HTML(f"<h3>{group} - {subgroup}</h3>"))
            display(HTML(top_n_weights_df.to_html(index=False)))

display_top_weights(price_indexes_secondary[end_year], top_n)

## Top Contributors to Index Change

In [ ]:
from IPython.display import display, HTML

def top_contributors_to_index_change(price_indexes, top_n=5, last_year=None):
    if last_year is None:
        last_year = max(price_indexes.keys())

    for group in price_indexes[last_year]:
        for subgroup in price_indexes[last_year][group]:
            # Calculate the contribution to the index change
            price_indexes[last_year][group][subgroup]['contribution'] = price_indexes[last_year][group][subgroup]['weight'] * price_indexes[last_year][group][subgroup]['price_ratio']

            # Sort by the contribution in descending order
            sorted_contributors = price_indexes[last_year][group][subgroup].sort_values(by='contribution', ascending=False)

            # Select the top n contributors
            top_contributors = sorted_contributors.head(top_n)

            # Display the results
            display(HTML(f"<h3>Top {top_n} Contributors to Index Change for {group} - {subgroup} in {last_year}</h3>"))
            display(HTML(top_contributors.to_html(index=False)))

top_contributors_to_index_change(price_indexes_secondary, top_n, last_year=end_year)

## Top Weight Differences

In [ ]:
def top_abs_weight_differences(weights, top_n=10):
    import matplotlib.pyplot as plt
    import pandas as pd
    from IPython.display import display, HTML

    for group in weights.keys():
        if group == 'All':
            continue

        subgroups = weights[group].keys()
        try:
            subgroups = sorted(subgroups, key=int)
        except ValueError:
            subgroups = weights[group].keys()
        n_subgroups = len(subgroups)
        ncols = 2
        nrows = (n_subgroups + ncols - 1) // ncols

        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10 * ncols, 5 * nrows), sharey=False)
        axes = axes.flatten()

        axis_max = 0
        axis_min = 0
        weights_diff_df = {}
        for i, subgroup in enumerate(subgroups):
            # Calculate the weight differences between the subgroup and the control group
            weights_diff_df[subgroup] = weights[group][subgroup].copy()
            weights_diff_df[subgroup]['weight_diff'] = weights_diff_df[subgroup]['weight'] - weights['All']['All']['weight']

            # Sort by the weight differences in descending order
            sorted_weights_diff_df = weights_diff_df[subgroup].sort_values(by='weight_diff', ascending=True)

            # Select the top n positive gaps
            top_n_positive = sorted_weights_diff_df.head(top_n)
            top_n_positive = top_n_positive.iloc[::-1]

            # Sort by the weight differences in ascending order
            sorted_weights_diff_df = weights_diff_df[subgroup].sort_values(by='weight_diff', ascending=False)

            # Select the top n negative gaps
            top_n_negative = sorted_weights_diff_df.head(top_n)

            # Concatenate the positive and negative gaps
            top_n_weights_diff_df = pd.concat([top_n_positive, top_n_negative])
            axis_max = max(axis_max, top_n_weights_diff_df['weight_diff'].max())
            axis_min = min(axis_min, top_n_weights_diff_df['weight_diff'].min())

            # Plot the top n largest gaps
            axes[i].barh(top_n_weights_diff_df['description'], top_n_weights_diff_df['weight_diff'], color='skyblue')
            axes[i].set_title(subgroup)
            axes[i].set_xlabel('Weight Difference')
            axes[i].set_ylabel('Description')
            axes[i].grid(True)

        for ax in axes[len(subgroups):]:
            fig.delaxes(ax)

        fig.suptitle(f"{group}: Top Weight Differences", fontsize=18)
        plt.tight_layout()
        plt.show()

top_abs_weight_differences(price_indexes_secondary[comparison_year], top_n)

In [ ]:
# Export to html (optional: --no-input)
!jupyter nbconvert --to html Main.ipynb --no-input --output Main.html